In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, BatchNormalization, Activation, Dense, Dropout, 
                                    Lambda, RepeatVector, Reshape, Conv2D, Conv2DTranspose,
                                    MaxPooling2D, GlobalMaxPool2D, concatenate, add, multiply,
                                    AveragePooling2D, UpSampling2D, Concatenate)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import cv2
from tqdm import tqdm
import warnings
import glob 
from datetime import datetime
import json 
from pathlib import Path
warnings.filterwarnings('ignore')
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
# ==================== SET CONFIGURATION PARAMETERS ====================

# Model parameters (must match your training configuration)
MAP_NAME = "Selworthy"

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
kinit = 'he_normal'

# File paths
MODEL_WEIGHTS_PATH = 'model-T_u-a_d_aspp-maproad.h5'

INPUT_FOLDER = f'tithe_patches/{MAP_NAME}/{MAP_NAME}_patches'
OUTPUT_FOLDER = f'tithe_patches/{MAP_NAME}//{MAP_NAME}_predictions'

# Create input and output folders if they do not exist
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Processing parameters
BATCH_SIZE = 10  # Adjust based on your GPU memory
THRESHOLD = 0.2  # Threshold for binary mask creation
SUPPORTED_FORMATS = ['.png', '.jpg', '.jpeg', '.tiff', '.tif', '.bmp']

# Output 
SAVE_MASKS = True
SAVE_OVERLAYS = False
SAVE_PROBABILITY_MAPS = False

In [3]:
# ==================== LOAD MODEL ARCHITECTURE FUNCTIONS ====================

# Loss functions
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def AttnGatingBlock(x, g, inter_shape, name):
    """Attention Gating Block"""
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(g)

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name)(x)
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),
                               strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                               padding='same', name='g_up'+name)(phi_g)

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)

    upsample_psi = Lambda(lambda x: K.repeat_elements(x, shape_x[3], axis=3), name='psi_up'+name)(upsample_psi)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same', name='q_attn_conv'+name)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm, name):
    """Gating signal for attention"""
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same", name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name=name + '_act')(x)
    return x

def UnetConv2D(input, outdim, is_batchnorm, name):
    """Basic U-Net convolution block"""
    x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=kinit, padding="same", name=name+'_1')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_act')(x)

    x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=kinit, padding="same", name=name+'_2')(x)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_act')(x)
    return x

def ASPP(input, out_channel, name):
    """Atrous Spatial Pyramid Pooling module"""
    # Branch 1: 1x1 convolution
    x1 = Conv2D(out_channel, (1, 1), kernel_initializer=kinit, padding="same", dilation_rate=1, name=name+'_conv1')(input)
    x1 = BatchNormalization(name=name+'_bn1')(x1)
    x1 = Activation('relu', name=name+'_act1')(x1)

    # Branch 2: 3x3 dilation rate 6
    x2 = Conv2D(out_channel, (3, 3), kernel_initializer=kinit, padding="same", dilation_rate=6, name=name+'_conv2')(input)
    x2 = BatchNormalization(name=name+'_bn2')(x2)
    x2 = Activation('relu', name=name+'_act2')(x2)

    # Branch 3: 3x3 dilation rate 12
    x3 = Conv2D(out_channel, (3, 3), kernel_initializer=kinit, padding="same", dilation_rate=12, name=name+'_conv3')(input)
    x3 = BatchNormalization(name=name+'_bn3')(x3)
    x3 = Activation('relu', name=name+'_act3')(x3)

    # Branch 4: 3x3 dilation rate 18
    x4 = Conv2D(out_channel, (3, 3), kernel_initializer=kinit, padding="same", dilation_rate=18, name=name+'_conv4')(input)
    x4 = BatchNormalization(name=name+'_bn4')(x4)
    x4 = Activation('relu', name=name+'_act4')(x4)

    # Branch 5: Global Average Pooling
    x5 = AveragePooling2D(pool_size=(1, 1))(input)
    x5 = Conv2D(out_channel, (1, 1), kernel_initializer=kinit, padding="same", name=name+'_conv5')(x5)
    x5 = BatchNormalization(name=name+'_bn5')(x5)
    x5 = Activation('relu', name=name+'_act5')(x5)
    x5 = UpSampling2D(size=(K.int_shape(input)[1]//K.int_shape(x5)[1], 
                          K.int_shape(input)[2]//K.int_shape(x5)[2]))(x5)

    # Concatenate all branches
    x = Concatenate(axis=3)([x1, x2, x3, x4, x5])
    x = Conv2D(out_channel, (1, 1), kernel_initializer=kinit, padding="same", name=name+'_conv_final')(x)
    x = BatchNormalization(name=name+'_bn_final')(x)
    x = Activation('relu', name=name+'_act_final')(x)
    x = Dropout(0.5)(x)

    return x

def build_attn_unet(input_size, loss_function):
    """Build the complete Attention U-Net with ASPP"""
    inputs = Input(shape=input_size)
    
    # Encoder
    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    conv1 = Dropout(0.2, name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = UnetConv2D(pool1, 32, is_batchnorm=True, name='conv2')
    conv2 = Dropout(0.2, name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = UnetConv2D(pool2, 64, is_batchnorm=True, name='conv3')
    conv3 = Dropout(0.2, name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = UnetConv2D(pool3, 64, is_batchnorm=True, name='conv4')
    conv4 = Dropout(0.2, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Center with ASPP
    center = ASPP(pool4, 128, name='center')
    
    # Decoder with Attention Gates
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', 
                                     activation='relu', kernel_initializer=kinit)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', 
                                     activation='relu', kernel_initializer=kinit)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up2, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', 
                                     activation='relu', kernel_initializer=kinit)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', 
                                     activation='relu', kernel_initializer=kinit)(up3), conv1], name='up4')
    
    # Output layer
    out = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=kinit, name='final')(up4)
    
    model = Model(inputs=[inputs], outputs=[out])
    return model

print("Model architecture functions loaded successfully!")

Model architecture functions loaded successfully!


In [4]:
# ==================== CREATE BATCH PROCESSING FUNCTION ====================

def load_and_preprocess_image(image_path):
    """Load and preprocess a single image for model input"""
    try:
        # Load image
        img = load_img(image_path, color_mode='rgb')
        original_size = img.size  # Store original size for later
        
        # Convert to array and resize
        img_array = img_to_array(img)
        img_resized = tf.image.resize(img_array, (IMG_HEIGHT, IMG_WIDTH))
        img_normalized = img_resized.numpy() / 255.0
        
        return img_normalized, img_array, original_size
    except Exception as e:
        print(f"Error loading image {image_path}: {str(e)}")
        return None, None, None

def postprocess_prediction(prediction, original_size, threshold=0.5):
    """Convert prediction to binary mask and resize to original dimensions"""
    # Create binary mask
    binary_mask = (prediction > threshold).astype(np.uint8)
    
    # Resize back to original size if needed
    if original_size != (IMG_WIDTH, IMG_HEIGHT):
        binary_mask_resized = tf.image.resize(
            binary_mask, original_size[::-1], method='nearest'
        ).numpy()
        prediction_resized = tf.image.resize(
            prediction, original_size[::-1]
        ).numpy()
    else:
        binary_mask_resized = binary_mask
        prediction_resized = prediction
    
    return binary_mask_resized, prediction_resized

def create_overlay(original_image, mask, alpha=0.3):
    """Create overlay of original image with predicted mask"""
    # Normalize original image if needed
    if original_image.max() > 1:
        original_image = original_image / 255.0
    
    # Create colored mask (red channel)
    colored_mask = np.zeros_like(original_image)
    colored_mask[:, :, 0] = mask.squeeze()  # Red channel
    
    # Create overlay
    overlay = original_image * (1 - alpha) + colored_mask * alpha
    return (overlay * 255).astype(np.uint8)

def process_batch(image_paths, model, output_dirs, progress_bar=None):
    """Process a batch of images"""
    batch_data = []
    valid_paths = []
    original_images = []
    original_sizes = []
    
    # Load and preprocess batch
    for img_path in image_paths:
        img_processed, img_original, orig_size = load_and_preprocess_image(img_path)
        if img_processed is not None:
            batch_data.append(img_processed)
            valid_paths.append(img_path)
            original_images.append(img_original)
            original_sizes.append(orig_size)
    
    if not batch_data:
        return []
    
    # Convert to numpy array for batch prediction
    batch_array = np.array(batch_data)
    
    # Make predictions
    predictions = model.predict(batch_array, verbose=0)
    
    # Process each prediction
    results = []
    for i, (pred, img_path, orig_img, orig_size) in enumerate(
        zip(predictions, valid_paths, original_images, original_sizes)
    ):
        try:
            # Get filename without extension
            filename = Path(img_path).stem
            
            # Postprocess prediction
            binary_mask, prob_map = postprocess_prediction(pred, orig_size, THRESHOLD)
            
            # Save binary mask
            if SAVE_MASKS:
                mask_path = os.path.join(output_dirs['masks'], f"{filename}_mask.png")
                mask_to_save = (binary_mask.squeeze() * 255).astype(np.uint8)
                cv2.imwrite(mask_path, mask_to_save)
            
            # Save probability map
            if SAVE_PROBABILITY_MAPS:
                prob_path = os.path.join(output_dirs['probability'], f"{filename}_prob.png")
                prob_to_save = (prob_map.squeeze() * 255).astype(np.uint8)
                cv2.imwrite(prob_path, prob_to_save)
            
            # Save overlay
            if SAVE_OVERLAYS:
                overlay = create_overlay(orig_img, binary_mask)
                overlay_path = os.path.join(output_dirs['overlays'], f"{filename}_overlay.png")
                cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
            
            results.append({
                'filename': filename,
                'original_path': img_path,
                'mask_path': mask_path if SAVE_MASKS else None,
                'overlay_path': overlay_path if SAVE_OVERLAYS else None,
                'processed': True
            })
            
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            results.append({
                'filename': Path(img_path).stem,
                'original_path': img_path,
                'processed': False,
                'error': str(e)
            })
        
        # Update progress bar
        if progress_bar:
            progress_bar.update(1)
    
    return results

print("Batch processing functions created")

Batch processing functions created


In [5]:
# ==================== SET UP INPUT AND OUTPUT DIRECTORIES ====================

def setup_directories():
    """Create output directories and scan input folder"""
    
    # Create output directories
    output_dirs = {
        'base': OUTPUT_FOLDER,
        'masks': os.path.join(OUTPUT_FOLDER, 'masks'),
        'overlays': os.path.join(OUTPUT_FOLDER, 'overlays'),
        'probability': os.path.join(OUTPUT_FOLDER, 'probability_maps')
    }
    
    for dir_path in output_dirs.values():
        os.makedirs(dir_path, exist_ok=True)
        print(f"Created directory: {dir_path}")
    
    # Scan input folder for images
    image_files = []
    if os.path.exists(INPUT_FOLDER):
        for ext in SUPPORTED_FORMATS:
            pattern = os.path.join(INPUT_FOLDER, f"*{ext}")
            image_files.extend(glob.glob(pattern))
            # Also check uppercase extensions
            pattern = os.path.join(INPUT_FOLDER, f"*{ext.upper()}")
            image_files.extend(glob.glob(pattern))
    
    image_files = sorted(list(set(image_files)))  # Remove duplicates and sort
    
    print(f"\nFound {len(image_files)} images in input folder:")
    print(f"Input folder: {INPUT_FOLDER}")
    
    if len(image_files) == 0:
        print("WARNING: No images found! Please check your input folder path and supported formats.")
        print(f"Supported formats: {SUPPORTED_FORMATS}")
    else:
        print(f"First few files: {[os.path.basename(f) for f in image_files[:5]]}")
        if len(image_files) > 5:
            print(f"... and {len(image_files) - 5} more files")
    
    return output_dirs, image_files

# Setup directories and get image list
output_directories, image_list = setup_directories()

print(f"\nSetup completed!")
print(f"Total images to process: {len(image_list)}")
print(f"Estimated batches: {(len(image_list) + BATCH_SIZE - 1) // BATCH_SIZE}")

Created directory: tithe_patches/Selworthy//Selworthy_predictions
Created directory: tithe_patches/Selworthy//Selworthy_predictions\masks
Created directory: tithe_patches/Selworthy//Selworthy_predictions\overlays
Created directory: tithe_patches/Selworthy//Selworthy_predictions\probability_maps

Found 3965 images in input folder:
Input folder: tithe_patches/Selworthy/Selworthy_patches
First few files: ['patch_0_0.png', 'patch_0_1.png', 'patch_0_10.png', 'patch_0_11.png', 'patch_0_12.png']
... and 3960 more files

Setup completed!
Total images to process: 3965
Estimated batches: 397


In [6]:
# ==================== PROCESS ALL IMAGES IN FOLDER ====================

def deploy_model_on_folder():
    """Main function to deploy model on entire folder"""
    
    if len(image_list) == 0:
        print("No images to process. Exiting.")
        return None
    
    # Load the trained model
    print("Loading trained model...")
    model = build_attn_unet((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dice_loss)
    
    if not os.path.exists(MODEL_WEIGHTS_PATH):
        print(f"ERROR: Model weights file not found: {MODEL_WEIGHTS_PATH}")
        return None
    
    model.load_weights(MODEL_WEIGHTS_PATH)
    print("Model loaded successfully!")
    
    # Process images in batches
    all_results = []
    total_images = len(image_list)

    print(f"\nStarting batch processing...")
    print(f"Total images: {total_images}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Total batches: {(total_images + BATCH_SIZE - 1) // BATCH_SIZE}")
    
    # Create progress bar
    with tqdm(total=total_images, desc="Processing images") as pbar:
        
        # Process in batches
        for i in range(0, total_images, BATCH_SIZE):
            batch_images = image_list[i:i + BATCH_SIZE]
            batch_num = (i // BATCH_SIZE) + 1
            total_batches = (total_images + BATCH_SIZE - 1) // BATCH_SIZE
            
            pbar.set_description(f"Processing batch {batch_num}/{total_batches}")
            
            try:
                # Process current batch
                batch_results = process_batch(batch_images, model, output_directories, pbar)
                all_results.extend(batch_results)
                
                # Clear some memory
                tf.keras.backend.clear_session()
                
            except Exception as e:
                print(f"\nError processing batch {batch_num}: {str(e)}")
                # Add failed results for this batch
                for img_path in batch_images:
                    all_results.append({
                        'filename': Path(img_path).stem,
                        'original_path': img_path,
                        'processed': False,
                        'error': str(e)
                    })
                pbar.update(len(batch_images))
    
    print("\nBatch processing completed!")
    return all_results, model

# Execute the deployment
print("=" * 50)
print("STARTING MODEL DEPLOYMENT")
print("=" * 50)

start_time = datetime.now()
results, trained_model = deploy_model_on_folder()
end_time = datetime.now()

processing_time = end_time - start_time
print(f"\nTotal processing time: {processing_time}")

if results:
    successful = sum(1 for r in results if r['processed'])
    failed = len(results) - successful
    print(f"Successfully processed: {successful}/{len(results)} images")
    if failed > 0:
        print(f"Failed to process: {failed} images")

STARTING MODEL DEPLOYMENT
Loading trained model...
Model loaded successfully!

Starting batch processing...
Total images: 3965
Batch size: 10
Total batches: 397


Processing batch 397/397: 100%|██████████| 3965/3965 [03:36<00:00, 18.29it/s]


Batch processing completed!

Total processing time: 0:03:39.327456
Successfully processed: 3965/3965 images


In [7]:
# ==================== SAVE RESULTS AND GENERATE SUMMARY ====================

def generate_summary_report(results, processing_time):
    """Generate a detailed summary report"""
    
    if not results:
        print("No results to summarize.")
        return
    
    # Calculate statistics
    total_images = len(results)
    successful = sum(1 for r in results if r['processed'])
    failed = total_images - successful
    success_rate = (successful / total_images) * 100 if total_images > 0 else 0
    
    # Create summary dictionary
    summary = {
        'processing_info': {
            'start_time': start_time.isoformat(),
            'end_time': end_time.isoformat(),
            'processing_time': str(processing_time),
            'processing_time_seconds': processing_time.total_seconds()
        },
        'configuration': {
            'model_weights_path': MODEL_WEIGHTS_PATH,
            'input_folder': INPUT_FOLDER,
            'output_folder': OUTPUT_FOLDER,
            'img_dimensions': f"{IMG_WIDTH}x{IMG_HEIGHT}",
            'batch_size': BATCH_SIZE,
            'threshold': THRESHOLD,
            'save_masks': SAVE_MASKS,
            'save_overlays': SAVE_OVERLAYS,
            'save_probability_maps': SAVE_PROBABILITY_MAPS
        },
        'results': {
            'total_images': total_images,
            'successful': successful,
            'failed': failed,
            'success_rate': round(success_rate, 2),
            'average_time_per_image': round(processing_time.total_seconds() / total_images, 2) if total_images > 0 else 0
        },
        'file_details': results
    }
    
    # Save summary as JSON
    summary_path = os.path.join(OUTPUT_FOLDER, 'processing_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    # Create and save text summary
    text_summary = f"""
    U-NET MODEL DEPLOYMENT SUMMARY
    =====================================
    
    Processing Information:
    - Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}
    - End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}
    - Total Processing Time: {processing_time}
    - Average Time per Image: {summary['results']['average_time_per_image']:.2f} seconds
    
    Configuration:
    - Model Weights: {MODEL_WEIGHTS_PATH}
    - Input Folder: {INPUT_FOLDER}
    - Output Folder: {OUTPUT_FOLDER}
    - Image Dimensions: {IMG_WIDTH}x{IMG_HEIGHT}
    - Batch Size: {BATCH_SIZE}
    - Threshold: {THRESHOLD}
    
    Results:
    - Total Images: {total_images}
    - Successfully Processed: {successful}
    - Failed: {failed}
    - Success Rate: {success_rate:.2f}%
    
    Output Files Generated:
    - Binary Masks: {'Yes' if SAVE_MASKS else 'No'}
    - Overlay Images: {'Yes' if SAVE_OVERLAYS else 'No'}
    - Probability Maps: {'Yes' if SAVE_PROBABILITY_MAPS else 'No'}
    """
    
    if failed > 0:
        text_summary += f"\n    Failed Files:\n"
        for result in results:
            if not result['processed']:
                text_summary += f"    - {result['filename']}: {result.get('error', 'Unknown error')}\n"
    
    # Save text summary
    text_summary_path = os.path.join(OUTPUT_FOLDER, 'processing_summary.txt')
    with open(text_summary_path, 'w') as f:
        f.write(text_summary)
    
    # Print summary to console
    print("\n" + "=" * 50)
    print("PROCESSING SUMMARY")
    print("=" * 50)
    print(text_summary)
    
    print(f"\nDetailed reports saved:")
    print(f"- JSON: {summary_path}")
    print(f"- Text: {text_summary_path}")
    
    return summary

# Generate summary report
if results:
    summary_data = generate_summary_report(results, processing_time)
    
    # Display some sample results
    print("\n" + "=" * 50)
    print("SAMPLE RESULTS")
    print("=" * 50)
    
    successful_results = [r for r in results if r['processed']]
    if successful_results:
        print("Successfully processed files (showing first 5):")
        for result in successful_results[:5]:
            print(f"✓ {result['filename']}")
            if result.get('mask_path'):
                print(f"  - Mask: {os.path.basename(result['mask_path'])}")
            if result.get('overlay_path'):
                print(f"  - Overlay: {os.path.basename(result['overlay_path'])}")
    
    failed_results = [r for r in results if not r['processed']]
    if failed_results:
        print(f"\nFailed files ({len(failed_results)} total):")
        for result in failed_results[:3]:  # Show first 3 failures
            print(f"✗ {result['filename']}: {result.get('error', 'Unknown error')}")
        if len(failed_results) > 3:
            print(f"  ... and {len(failed_results) - 3} more failures")

# Clean up memory
if 'trained_model' in locals():
    del trained_model
tf.keras.backend.clear_session()

print("\n" + "=" * 50)
print("DEPLOYMENT COMPLETED SUCCESSFULLY!")
print("=" * 50)
print(f"Check your output folder: {OUTPUT_FOLDER}")


PROCESSING SUMMARY

    U-NET MODEL DEPLOYMENT SUMMARY
    
    Processing Information:
    - Start Time: 2025-09-23 16:51:44
    - End Time: 2025-09-23 16:55:24
    - Total Processing Time: 0:03:39.327456
    - Average Time per Image: 0.06 seconds
    
    Configuration:
    - Model Weights: model-T_u-a_d_aspp-maproad.h5
    - Input Folder: tithe_patches/Selworthy/Selworthy_patches
    - Output Folder: tithe_patches/Selworthy//Selworthy_predictions
    - Image Dimensions: 256x256
    - Batch Size: 10
    - Threshold: 0.2
    
    Results:
    - Total Images: 3965
    - Successfully Processed: 3965
    - Failed: 0
    - Success Rate: 100.00%
    
    Output Files Generated:
    - Binary Masks: Yes
    - Overlay Images: No
    - Probability Maps: No
    

Detailed reports saved:
- JSON: tithe_patches/Selworthy//Selworthy_predictions\processing_summary.json
- Text: tithe_patches/Selworthy//Selworthy_predictions\processing_summary.txt

SAMPLE RESULTS
Successfully processed files (showing